In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.nlp import *
from sklearn.linear_model import LogisticRegression

In [2]:
PATH='D:/FAST.Ai/Machine Learning for coders/Data/_aclmdb2/'
names = ['neg','pos']

In [3]:
trn,trn_y = texts_labels_from_folders(f'{PATH}train',names)
val,val_y = texts_labels_from_folders(f'{PATH}test',names)

In [4]:
len(trn)

2102

In [47]:
# what is movie review

In [45]:
trn[0]

"Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly."

In [46]:
trn_y[0]

0

In [5]:
veczr = CountVectorizer(tokenizer=tokenize)

In [4]:
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

NameError: name 'veczr' is not defined

In [7]:
trn_term_doc

<2102x23843 sparse matrix of type '<class 'numpy.int64'>'
	with 313111 stored elements in Compressed Sparse Row format>

In [8]:
trn_term_doc[0]

<1x23843 sparse matrix of type '<class 'numpy.int64'>'
	with 93 stored elements in Compressed Sparse Row format>

In [9]:
vocab = veczr.get_feature_names() ; vocab[1000:1010]

['amazes',
 'amazing',
 'amazingly',
 'amazon',
 'amazonas',
 'amazonians',
 'ambassador',
 'ambiance',
 'ambidexterous',
 'ambiguities']

In [10]:
# set make them unique 
w0 = set([o.lower() for o in trn[0].split(" ")]); w0

{'a',
 'absurd',
 'an',
 'and',
 'audience',
 'be',
 'better',
 'briefly.',
 'by',
 'can',
 'chantings',
 'cinematography',
 'comedy.',
 'crazy',
 'cryptic',
 'dialogue',
 'easy',
 'era',
 'even',
 'eventually',
 'example',
 'feelings',
 'for',
 'formal',
 'forrest',
 'frederic',
 'from',
 'future',
 'general',
 'good',
 'grader.',
 'great',
 'has',
 'insane,',
 'into',
 'is',
 'it',
 "it's",
 'just',
 'kirkland',
 'level',
 'make',
 'making',
 'man',
 'might',
 'mob',
 'narrative',
 'no',
 'of',
 'off',
 'off.',
 'on',
 'opening',
 'orchestra',
 'out',
 'pig.',
 'putting.',
 'sally',
 'scene',
 'seem',
 'seen',
 'shakespeare',
 'should',
 'singers.',
 'some',
 'stars',
 'starts',
 'stays',
 'story',
 'technical',
 'terrific',
 'than',
 'that',
 'the',
 'think',
 'third',
 'those',
 'time',
 'to',
 'too',
 'turned',
 'unfortunately',
 'unnatural',
 'vilmos',
 'violent',
 'who',
 'whole',
 'with',
 'would',
 'you',
 'zsigmond.'}

In [11]:
len(w0)

91

In [12]:
veczr.vocabulary_["absurd"]

430

In [13]:
trn_term_doc[0,430]

2

In [14]:
trn_term_doc[0,410]

0

In [15]:
vocab[410]

'abrasive'

# Naive Bayes

In [15]:
def pr(y_i):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [120]:
x=trn_term_doc
y=trn_y

r = np.log(pr(1)/pr(0))
b = np.log((y==1).mean() / (y==0).mean())

In [121]:
r

matrix([[-0.06634, -0.07451, -0.15765, ...,  0.59802,  0.59802,  0.59802]])

In [122]:
pre_preds = val_term_doc @ r.T + b
preds = pre_preds.T > 0

In [123]:
(preds == val_y).mean()

0.7808764940239044

In [117]:
# binarized Naive Bayes

x = trn_term_doc.sign()
y = trn_y

p = x[y == 1].sum(0) +1 
q = x[y == 0].sum(0) +1 

r = np.log((p/((y==1).sum()+1))/(q((y==0).sum()+1)))
b = np.log(((y==1).mean() +1) / ((y==0).mean() +1))

print((y==1).sum(),"\n")

pre_preds = val_term_doc.sign() @ r.T + b
preds = pre_preds.T > 0
(preds == val_y).mean()

1101 



0.8007968127490039

In [118]:
# i write this wrong thing at first and i don't know why is that better :)

x = trn_term_doc.sign()
y = trn_y

p = x[y == 1].sum(0) +1 
q = x[y == 0].sum(0) +1 

r = np.log((p/len(p))/(q/len(q)))
b = np.log(len(p) / len(q))

print(len(p),"\t",len(q),"\n")

pre_preds = val_term_doc.sign() @ r.T + b
preds = pre_preds.T > 0
(preds == val_y).mean()

1 	 1 



0.8346613545816733

# Logistic Regresion

In [29]:
m = LogisticRegression(C=1e8 , dual=True)

x = trn_term_doc
y = trn_y

m.fit(x,y)
preds = m.predict(val_term_doc)
(preds == val_y).mean()

C:\Users\Lenovo\Anaconda3\envs\fastai\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Lenovo\Anaconda3\envs\fastai\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8227091633466136

In [31]:
# binarized Naive Bayes

m = LogisticRegression(C=1e8 , dual=True)

x = trn_term_doc.sign()
y = trn_y

m.fit(x,y)
preds = m.predict(val_term_doc.sign())
(preds == val_y).mean()

C:\Users\Lenovo\Anaconda3\envs\fastai\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Lenovo\Anaconda3\envs\fastai\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8286852589641435

# with regularization 

In [32]:
# binarized Naive Bayes

m = LogisticRegression(C=1 , dual=True)

x = trn_term_doc.sign()
y = trn_y

m.fit(x,y)
preds = m.predict(val_term_doc.sign())
(preds == val_y).mean()

C:\Users\Lenovo\Anaconda3\envs\fastai\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8446215139442231

# n grams 

In [9]:
veczr = CountVectorizer(ngram_range=(1,3) , tokenizer=tokenize , max_features=800000)
# by saying "max_features=800000" countVectorizer will sort the vocabulary by how often each feacher 
    #appeard and will remove which less appeard , if features were more than num of max_features
# for "ngram_range" >>    min_n <= n <= max_n

trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [10]:
trn_term_doc.shape

(2102, 652953)

In [11]:
vocab = veczr.get_feature_names()
vocab[1000:1010]

['! thanks ,',
 '! thanks again',
 '! that',
 '! that has',
 '! that is',
 '! that latter',
 '! that made',
 '! that makes',
 '! that scene',
 '! the']

In [12]:
m = LogisticRegression(C=1 , dual=True)

x = trn_term_doc.sign()
y = trn_y

m.fit(x,y)
preds = m.predict(val_term_doc.sign())
(preds == val_y).mean()

C:\Users\Lenovo\Anaconda3\envs\fastai\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8605577689243028

# combination of Naive Bayes and Logistic Regression

In [13]:
y=trn_y
x=trn_term_doc.sign()
val_x = val_term_doc.sign()

In [16]:
r = np.log(pr(1) / pr(0))
b = np.log((y==1).mean() / (y==0).mean())

In [17]:
x_nb = x.multiply(r) # equal to "x * r " which was for numpy but as if "*" here means matmul 

In [18]:
x

<2102x652953 sparse matrix of type '<class 'numpy.int64'>'
	with 1429342 stored elements in Compressed Sparse Row format>

In [19]:
r.shape

(1, 652953)

In [20]:
x_nb.shape , type(x_nb)

((2102, 652953), scipy.sparse.coo.coo_matrix)

In [21]:
m = LogisticRegression(dual=True , C=0.1)
m.fit(x_nb , y)

val_x_nb = val_x.multiply(r)
preds = m.predict(val_x_nb)
(preds == val_y).mean()

0.8844621513944223

# fastai NBSVM++

In [22]:
sl = 2000 # number of unique words per each review 

In [23]:
md = TextClassifierData.from_bow(trn_term_doc,trn_y,val_term_doc,val_y ,sl )

In [25]:
learner = md.dotprod_nb_learner()
learner.fit(0.02 , n_cycle = 1 , wds = 1e-6 ,cycle_len=1)

epoch      trn_loss   val_loss   <lambda>                                                                              
    0      0.001079   0.214447   0.840637  



[array([0.21445]), 0.8406374501992032]

In [26]:
learner = md.dotprod_nb_learner()
learner.fit(0.02 , n_cycle = 4 , wds = 1e-6 ,cycle_len=1)

epoch      trn_loss   val_loss   <lambda>                                                                              
    0      0.001111   0.212721   0.846614  
    1      0.00079    0.201115   0.86255                                                                               
    2      0.000572   0.198086   0.870518                                                                              
    3      0.00044    0.195522   0.868526                                                                              



[array([0.19552]), 0.8685258964143426]